Este tutorial mostra os detalhes da construção de um recomendador de conteúdos baseado em Filtragem Colaborativa. Em nossos experimentos utilizaremos os dados fornecidos pela MovieLens, um dataset gratuito criado e mantido pelo grupo de pesquisas em recomendação de conteúdos da Universidade de Minnesota (GroupLens).

Estes dados podem ser obtidos a partir do seguinte endereço:

http://files.grouplens.org/datasets/movielens/ml-100k.zip

Veja a seguir a configuração dos dados:

In [1]:
import numpy  as np
import pandas as pd

names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


O arquivo "u.data", apresentado acima, possui 100 mil registros que representam o consumo e avaliação de um filme por um usuário. Cada registro é composto por: identificador do usuário, identificador do item, avalição e data / hora do registro.

A seguir, apresentamos o total de usuários e filmes avaliados:

In [2]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print str(n_users) + ' users'
print str(n_items) + ' items'

943 users
1682 items


Para facilitar os cálculos vamos organizar os dados em uma matriz, onde as linhas correspondem aos usuário e as colunas correspondem aos itens. Os valores de cada coluna será preenchido pela nota dada ao item, por cada usuários. Os itens não avaliados por um usuário reveberão o valor 0 (zero).

Outra alternativa é criar uma matriz booleana, ignorando as notas dadas pelos usuários e indicando apenas se o usuário consumiu o conteúdo.

Veja:

In [3]:
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
    #ratings[row[1]-1, row[2]-1] = 1
ratings

array([[ 5.,  3.,  4., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.]])

Embora tenhamos preenchido as classificações faltantes como 0 (zero), estes valores deveriam ser vistos como entradas vazias, pois são os elementos que nosso algoritmo deverá predizer. Essa predição depende do cálculo de similaridade entre os usuários e os itens, que toma como base seu histórico de consumo.

Existem diversas medidas para o cálculo da similidade, a mais comum é a distância de cosenos. O resultado dado pela distância de cosenos é um ângulo entre dois vetores, quanto maior o ângulo mais distintos são os usuários / itens.

A seguir, a medida de distância por cosenos:

$$ \mathrm{sim(u, k)=\displaystyle\sum_{i=1}^{n}\frac{r_{ui}.r_{ki}}{\sqrt{\sum_{j=1}^{n} r_{uj}^{n}} \sqrt{\sum_{j=1}^{n} r_{kj}^{n}}}} $$

In [4]:
def similarity(ratings, kind='user', epsilon=1e-9):
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

É necessário considerar um fator epsilon para que não haja erros de divisões por zero, uma vez que a matriz de avaliações possui valores nulos. Para não divergir muito nos resultados, o valor de epsilon deve ser o mais próximo de zero possível.

Uma vez definida a função de similaridade  podemos criar a função de predição. A função de predição é responsável por atribuir uma classificação para os itens que cada usuário ainda não avaliou, utilizando como base a relação de semelhança entre os usuários / itens.

Veja:

In [5]:
def predict(ratings, similarity, kind='user'):
    if kind == 'user':
        return similarity.dot(ratings) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif kind == 'item':
        return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

Para avaliar a capacidade de predição do nosso algoritmo vamos separar os dados em dois conjuntos, treino e teste. Para isso, vamos colocar aproximadamente 10 classificações dos usuários em cada conjunto.

In [6]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in xrange(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size=10, replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]

    assert(np.all((train * test) == 0))
    return train, test

train, test = train_test_split(ratings)

É importante lembrar que os conjuntos de treino e teste devem ser disjuntos, ou seja, eles não devem conter elementos em comum. Uma forma de validar se dois conjuntos são disjuntos é mostrar que intersecção de ambos resulta em um conjunto vazio.

Para avaliar a eficiência do preditor, devemos treiná-lo com os dados de treinamento e utilizar os dados de teste para calcular a diferença entre as recomendações esperadas e as obtidas.

Veja:

In [7]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # ignorar termos nao classificados.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

user_similarity = similarity(train, kind='user')
item_similarity = similarity(train, kind='item')

item_prediction = predict(train, item_similarity, kind='item')
user_prediction = predict(train, user_similarity, kind='user')

print 'User-based MSE: ' + str(get_mse(user_prediction, test))
print 'Item-based MSE: ' + str(get_mse(item_prediction, test))

User-based MSE: 8.40126218564
Item-based MSE: 11.5537618551


Podemos minimizar o erro quadrático médio se considerar-mos os k usuários / itens mais parecidos entre si, ou seja, os top k elementos recomendados. É importante observar que tanto valores muito pequenos quanto valores muito grandes para k, podem produzir erros quadráticos médios grandes. Isso acontece porque o top k elementos pode não incluir as recomendações esperadas, como podem incluir muitas recomendações inesperadas.

Veja o próximo experimento, para k = 40:

In [8]:
def predict_topk(ratings, similarity, kind='user', k=40):
    pred = np.zeros(ratings.shape)
    if kind == 'user':
        for i in xrange(ratings.shape[0]):
            top_k_users = [np.argsort(similarity[:,i])[:-k-1:-1]]
            for j in xrange(ratings.shape[1]):
                pred[i, j] = similarity[i, :][top_k_users].dot(ratings[:, j][top_k_users]) 
                pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))
    if kind == 'item':
        for j in xrange(ratings.shape[1]):
            top_k_items = [np.argsort(similarity[:,j])[:-k-1:-1]]
            for i in xrange(ratings.shape[0]):
                pred[i, j] = similarity[j, :][top_k_items].dot(ratings[i, :][top_k_items].T) 
                pred[i, j] /= np.sum(np.abs(similarity[j, :][top_k_items]))        
    
    return pred

pred = predict_topk(train, user_similarity, kind='user', k=40)
print 'Top-k User-based MSE: ' + str(get_mse(pred, test))

pred = predict_topk(train, item_similarity, kind='item', k=40)
print 'Top-k Item-based MSE: ' + str(get_mse(pred, test))

Top-k User-based MSE: 6.49493678431
Top-k Item-based MSE: 7.7354164236


É importante mencionar que a função "predict_topk" não pode ser resolvida somente com operações matriciais, de modo que é necessário percorrer a matriz para calcular a similaridade entre os elementos e descobrir os k primeiros. Por conta disso a performance é muito ruim, se comparada a função "predict", que utiliza apenas operações matriciais.

Embora os erros quadráticos médios sejam pequenos, é importante validar o classificador através de algumas simulações. Como estamos utilizando uma base de dados de filmes em nossos experimentos, podemos fazer essa análise de forma intuitiva.

A seguir, veja as informações que temos sobre os filmes:

In [9]:
!head -5 u.item

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0


Observe a base de dados de filmes possui uma URL para o IMDB do filme, de onde podemos extrair o poster oficial e simular algumas situações.

Veja:

In [10]:
import warnings
import requests
import json

# ignore iPython warnings.
warnings.filterwarnings('ignore')

def get_poster(url):
    response = requests.get(url)
    movie_id = response.url.split('/')[-2]
    
    # read poster path.
    response = requests.get('https://api.themoviedb.org/3/movie/%s?api_key=63e2d5374176e0ea2e8fa271de9b6fda' % (movie_id))
    datapath = json.loads(response.text)
    if not 'poster_path' in datapath:
        print '[ERROR] %s - URL not found.' % (url)
        return None
    
    postpath = datapath['poster_path']
    # return poster url.
    response = requests.get('http://image.tmdb.org/t/p/original%s' % (postpath))
    return 'http://image.tmdb.org/t/p/original%s' % (postpath)

Algumas URL's podem não ser encontradas, por pequenas divergências ou alterações no endereçamento. Nestes casos apresentamos os posters que foram encontrados e listamos as URL's dos posters que divergirem.

Para simular uma recomendação, criamos uma função que retorne os k itens mais recomendados, com k inicial = 6. Também criamos uma função que exibe os posters dos filmes, se encontrados.

Veja:

In [11]:
from IPython.display import Image
from IPython.display import display

idx_to_movie = {}
with open('u.item', 'r') as f:
    for line in f.readlines():
        info = line.split('|')
        idx_to_movie[int(info[0])-1] = info[4]

def top_k_movies(similarity, mapper, movie_idx, k=6):
    return [mapper[x] for x in np.argsort(similarity[movie_idx,:])[:-k-1:-1]]

def show_posters(idx, similarity):
    movies = top_k_movies(similarity, idx_to_movie, idx)
    posters = []
    for movie in movies:
        poster = get_poster(movie)
        if poster:
            posters.append(Image(url=poster, width=150, height=150))

    display(*tuple(posters))

Para a primeira simulação selecionamos o filme infantil Aladdin. Como podemos observar a seguir, uma análise intuitiva sugere que o conteúdo recomendado está próximo ao esperado. Também avaliamos as recomendações para o filme Toy Story, e os resultados foram satisfatórios.

Veja:

In [38]:
idx = 94 # Aladdin
show_posters(idx, item_similarity)

[ERROR] http://us.imdb.com/M/title-exact?Lion%20King,%20The%20(1994) - URL not found.
[ERROR] http://us.imdb.com/M/title-exact?Empire%20Strikes%20Back,%20The%20(1980) - URL not found.


Nessa outra simulação selecionamos um filme de ação (GoldenEye) e após avaliar os resultados consideramos que as recomendações também foram satisfatórias. Também testamos essa situação com o filme "Terminator", onde o resultado também foi como o esperado.

Veja:

In [13]:
idx = 1 # GoldenEye
show_posters(idx, item_similarity)

[ERROR] http://us.imdb.com/M/title-exact?Speed%20(1994/I) - URL not found.


Em nosso próximo experimento selecionamos uma "comédia tosca", e como esperado todos os filmes recomendados são comédias da mesma categoria.

Veja:

In [14]:
idx = 40 # Billy Madison
show_posters(idx, item_similarity)

Um ponto que acreditamos ser muito sensível é a medida de similaridade. Nos experimentos anteriores utilizamos uma distãncia de cossenos, que mede o ângulo de diferenciação entre dois conteúdos. Para avaliar o quanto as medidas de similaridade podem influenciar nas recomendações, resolvemos repetir os experimentos utilizando os Coeficientes de Pearson, uma medida de correlação linear.

$$ \mathrm{sim(u, k)=\displaystyle\frac{\sum_{i=1}^{n} (x_{i} - \bar{x})(k_{i} - \bar{k})}{\sqrt{\sum_{i=1}^{n} (x_{i} - \bar{x})^2} \sqrt{\sum_{j=1}^{n} (k_{i} - \bar{k})}}} = \frac{cov(x, k)}{\sqrt{var(x).var(k)}}$$

Veja:

In [15]:
from sklearn.metrics import pairwise_distances
# pearson coefficient
item_correlation = 1 - pairwise_distances(train.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0.

Reproduzindo o primeiro experimento utilizando os Coeficientes de Pearson:

In [16]:
idx = 94 # Aladdin
show_posters(idx, item_correlation)

[ERROR] http://us.imdb.com/M/title-exact?Lion%20King,%20The%20(1994) - URL not found.


Reproduzindo o segundo experimento com os Coeficientes de Pearson:

In [17]:
idx = 1 # GoldenEye
show_posters(idx, item_correlation)

[ERROR] http://us.imdb.com/M/title-exact?Speed%20(1994/I) - URL not found.


Reproduzindo o terceiro experimento com os Coeficientes de Pearson:

In [19]:
idx = 40 # Billy Madison
show_posters(idx, item_correlation)

Conclusões:

* Executamos os experimentos em relação as similidades dos itens, pois assim seria mais fácil avaliar, visto que não conhecemos os usuários.
* O conjunto de dados utilizado apresentou pouca variação entre a matriz de notas e a matriz de visitas, embora o erro quadrático médio de ambas as situações tenham sido muito diferentes.
* Ambas as medidas de semelhança utilizadas produziram resultados satisfatórios.
* A transformação dos algoritmos em operações matriciais é muito mais performático.